In [ ]:
stop_words_remove = [
 'bà',
 'bài',
 'bán',
 'bản_thân',
 'bắt_đầu',
 'chơi',
 'chắc_chắn',
 'chị',
 'chịu',
 'chọn',
 'cá_nhân',
 'cơ_hội',
 'cảm_thấy',
 'cậu',
 'dài',
 'dành',
 'dễ',
 'em',
 'giá_trị',
 'giữ',
 'gần',
 'gặp',
 'hiện_tại',
 'hoàn_toàn',
 'hỏi',
 'khá',
 'khó',
 'khó_khăn',
 'kể',
 'lâu',
 'lòng',
 'lý_do',
 'lớn',
 'lời',
 'mở',
 'nghe',
 'nghĩ',
 'nguồn',
 'ngồi',
 'nhanh',
 'nhà',
 'nhìn',
 'nhóm',
 'nhất',
 'nhận',
 'nhỏ',
 'nhớ',
 'nhờ',
 'nước',
 'quan_trọng',
 'quan_tâm',
 'quay',
 'quá_trình',
 'sáng',
 'thay_đổi',
 'tháng',
 'thêm',
 'thích',
 'thậm_chí',
 'thật',
 'thời_gian',
 'thời_điểm',
 'thực_hiện',
 'thực_tế',
 'tin',
 'tiếp_tục',
 'trở_thành',
 'tìm',
 'tính',
 'tăng',
 'tạo',
 'tập_trung',
 'tốt',
 'tự',
 'việc',
 'vượt',
 'vấn_đề',
 'xa',
 'xem',
 'xin',
 'xuất_hiện',
 'ăn',
 'điểm',
 'đúng',
 'đạt',
 'đầu_tiên',
 'đặc_biệt',
 'đặt',
 'đủ',
 'chia_sẻ'
 ]

In [ ]:
stop_words_add = ['đi',
 'ta',
 '1',
 'the',
 '2',
 'hai',
 'chẳng',
 'câu',
 '3',
 'a',
 'of',
 'đấy',
 'and',
 'đứa',
 'mấy',
 'thành',
 '4',
 'lắm',
 'i',
 '5',
 'đừng',
 'in',
 'có_lẽ',
 'kẻ',
 'kia',
 'you',
 '10',
 'hắn',
 'kiểu',
 '’',
 's',
 'is',
 'bảo',
 'xảy',
 'nỗi',
 'thằng',
 'chỗ',
 '6',
 'it',
 'xong',
 'that',
 'hôm',
 'hôm_nay',
 'coi',
 'hơi',
 'dần',
 '7',
 'là',
 'suốt',
 't',
 'mày',
 'chút',
 'liệu',
 'có',
 'vốn',
 'đem',
 'bọn',
 'và',
 'thay',
 'for',
 'một',
 '8',
 'mãi',
 '\xa0',
 'tui',
 'tao',
 'href',
 'class',
 'target',
 'blank',
 'www',
 'thumb',
 'scrape',
 'rel',
 'image',
 'nofollow',
 'dofollow']

In [ ]:
import pandas as pd
import random 
import numpy as np
import math
import gensim
# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Spiderum/Data

[Errno 2] No such file or directory: '/content/drive/MyDrive/Spiderum/Data'
/content


In [ ]:
posts = pd.read_csv('./posts_cleaned.csv')
posts.head()

,_id,type,status,reading_time,tags.0,tags.1,tags.2,tags.3,tags.4,power_point,point,up_point,down_point,date_point,hot_point,controlversial_point,star,views_count,comment_count,is_edit,public_post,title,body,cat_id,creator_id,new_title,slug,created_at,updated_at,__v,new_id
0,6044ed3ad88da110e69300f7,0,0,4.8,NaN,NaN,NaN,NaN,NaN,0,25,25,0,1.452156e+12,535.064607,865.540000,False,122,0,0,False,Chào mừng tới spiderum,"<div>Hello, đây là bài post đầu tiên của spide...",1.0,6044501c1dc0770a602127fa,NaN,Chao-mung-toi-spiderum-1,2016-03-06T23:00:00.000Z,2021-03-07T15:11:54.483Z,0,6044ed3ad88da110e69300f7
1,6044ed3ad88da110e69300f8,0,1,50.4,NaN,NaN,NaN,NaN,NaN,0,17,17,0,1.452592e+12,534.897116,892.540000,False,4610,27,0,False,Chuyện tình đẫm lệ :(,"<div>Một lần can ngăn bạn bè xô xát, anh vô ý ...",2.0,6044501c1dc0770a602127fb,NaN,Chuyen-tinh-dam-le-2,2016-03-06T23:00:00.000Z,2021-03-07T15:11:54.491Z,1,6044ed3ad88da110e69300f8
2,6044ed3ad88da110e69300f9,0,1,90.8,NaN,NaN,NaN,NaN,NaN,0,129,129,0,1.452592e+12,535.777256,909.540000,False,9422,44,0,False,Tình người,<div>Ông là tổng giám đốc một tập đoàn lớn nhấ...,2.0,6044501c1dc0770a602127fc,NaN,Tinh-nguoi-3,2016-03-06T23:00:00.000Z,2021-03-07T15:11:54.495Z,1,6044ed3ad88da110e69300f9
3,6044ed3ad88da110e69300fa,1,1,5.2,NaN,NaN,NaN,NaN,NaN,0,24,24,0,1.453693e+12,535.046878,923.540000,False,7015,58,0,False,Dành cho những bạn đam mê khởi nghiệp,http://cafebiz.vn/xu-huong-cong-nghe/khi-gia-n...,8.0,6044501c1dc0770a6021280a,NaN,Danh-cho-nhung-ban-dam-me-khoi-nghiep-4,2016-03-06T23:00:00.000Z,2021-03-07T15:11:54.499Z,1,6044ed3ad88da110e69300fa
4,6044ed3ad88da110e69300fb,1,1,2.6,NaN,NaN,NaN,NaN,NaN,0,23,24,1,1.453695e+12,535.028394,911.677411,False,5348,46,0,False,Nghệ thuật đàm phán lương.,http://ybox.vn/nghe-nghiep/10-loi-khuyen-khi-d...,8.0,6044501c1dc0770a60212817,NaN,Nghe-thuat-dam-phan-luong-5,2016-03-06T23:00:00.000Z,2021-03-07T15:11:54.503Z,1,6044ed3ad88da110e69300fb


In [ ]:
# Drop NA values of body col
posts.dropna(subset=['body'], inplace=True)
posts.reset_index(drop=True, inplace=True)

In [ ]:
# Define a function to clean text
import re
import string
puncs = string.punctuation + '“”–…'

def clean_body(post):
    clean_post = re.sub(r'http\S+', '', post) # Clean any url link
    clean_tag_regex = re.compile('<.*?>')
    clean_post = re.sub(clean_tag_regex, ' ', clean_post) # Clean any tag <>
    clean_post = clean_post.lower()
    clean_post = clean_post.replace('nbsp', ' ')
    for punc in puncs:
        clean_post = clean_post.replace(punc, ' ')
    return clean_post

In [ ]:
# Clean body
posts['cleaned_body'] = posts['title'].apply(lambda x: clean_body(x)) + ' ' + posts['body'].apply(lambda x: clean_body(x))

# Tokenize

In [ ]:
!pip3 install vncorenlp
!mkdir -p vncorenlp/models/wordsegmenter
!wget -q --show-progress https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget -q --show-progress https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget -q --show-progress https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

     |████████████████████████████████| 2.7MB 5.7MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-cp37-none-any.whl size=2645936 sha256=5ec197af979699bd97a6800a8cdbd70dfd2dd826e915556b635105c488bb21ef
  Stored in directory: /root/.cache/pip/wheels/09/54/8b/043667de6091d06a381d7745f44174504a9a4a56ecc9380c54
Successfully built vncorenlp
VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  46.4MB/s    in 0.6s    
vi-vocab            100%[===================>] 514.20K  --.-KB/s    in 0.06s   
wordsegmenter.rdr   100%[===================>] 125.50K  --.-KB/s    in 0.03s   


In [ ]:
from vncorenlp import VnCoreNLP
tokenizer = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [ ]:
# Tokenization
def tokenize(x):
    res = ''
    try:
        res = tokenizer.tokenize(x)[0]
    except:
        res = []
    return res

posts['tokenized_body'] = posts['cleaned_body'].apply(lambda x: tokenize(x))

In [ ]:
CB_tokenized = posts[['slug', 'tokenized_body']]
CB_tokenized

In [ ]:
CB_tokenized.to_csv('./Data for CB/CB_tokenized.csv', index=False)

# LDA Model

In [ ]:
tokenize_df = pd.read_csv('./Data for CB/CB_tokenized.csv')
tokenize_df.head()

,slug,tokenized_body
0,Chao-mung-toi-spiderum-1,"['chào_mừng', 'tới', 'spiderum', 'hello', 'đây..."
1,Chuyen-tinh-dam-le-2,"['chuyện', 'tình', 'đẫm', 'lệ', 'một', 'lần', ..."
2,Tinh-nguoi-3,"['tình', 'người', 'ông', 'là', 'tổng_giám_đốc'..."
3,Danh-cho-nhung-ban-dam-me-khoi-nghiep-4,"['dành', 'cho', 'những', 'bạn', 'đam_mê', 'khở..."
4,Nghe-thuat-dam-phan-luong-5,"['nghệ_thuật', 'đàm_phán', 'lương']"


## Build model by LDA Mallet

In [ ]:
from gensim.models.ldamodel import LdaModel
def convertldaMalletToldaGen(mallet_model):
    model_gensim = LdaModel(
        id2word=mallet_model.id2word, num_topics=mallet_model.num_topics,
        alpha=mallet_model.alpha) # original function has 'eta=0' argument
    model_gensim.state.sstats[...] = mallet_model.wordtopics
    model_gensim.sync_state()
    return model_gensim

In [ ]:
import pyLDAvis.gensim_models as gensimvis

vis_data = gensimvis.prepare(ldagensim, a.BoW_corpus, a.dictionary, sort_topics=False)
pyLDAvis.display(vis_data)

In [ ]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip
!pip install pyLDAvis

In [ ]:
import ast
from gensim import corpora
import collections
from pprint import pprint
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from wordcloud import WordCloud
import pickle
from gensim.models.ldamodel import LdaModel

class LDAmodel(object):
    def __init__(self, tokenize_df, stop_words_add, stop_words_remove):
        self.tokenize_df = tokenize_df
        self.mallet_path = './mallet-2.0.8/bin/mallet'
        self.stop_words_add = stop_words_add
        self.stop_words_remove = stop_words_remove
        self.final_stop_words = None
        self.BoW_corpus = None
        self.dictionary = None
        self.ldamallet = None
        self.name_topics = None

    def clean_docs(self):
        # Drop duplicates
        self.tokenize_df.drop_duplicates(subset=['tokenized_body'], inplace=True)
        # Because list data convert into string when saving, so we need to convert back to list
        self.tokenize_df['tokenized_body'] = self.tokenize_df['tokenized_body'].apply(lambda x: ast.literal_eval(x))
    
    def remove_stop_words(self):
        # Create dictionary and Bag of Words
        self.dictionary = corpora.Dictionary(self.tokenize_df['tokenized_body'])
        self.BoW_corpus = [self.dictionary.doc2bow(doc) for doc in self.tokenize_df['tokenized_body']]
        
        # Filter top 500 words that show up the most
        BoW_dict = collections.defaultdict(int)
        for doc in self.BoW_corpus:
            for id, freq in doc:
                BoW_dict[self.dictionary[id]] += freq

        BoW_dict = pd.Series(BoW_dict).reset_index()
        BoW_dict.columns = ['word', 'count']
        BoW_dict.sort_values(by='count', ascending=False, inplace=True)

        top_tf_words = list(BoW_dict.iloc[:500, :]['word'])

        # List stop word from GitHub
        stopwords_VN = pd.read_csv('https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords-dash.txt', delimiter = "\t", header=None)
        stopwords_VN = list(stopwords_VN.iloc[:, 0])

        # Make a final list of stop words
        stop_words = [w for w in stopwords_VN if w in top_tf_words]
        self.final_stop_words = list(set(stop_words + self.stop_words_add) - set(self.stop_words_remove))

        # Filter stop words and extreme words
        self.tokenize_df['tokenized_body'] = self.tokenize_df['tokenized_body'].apply(lambda x: [w for w in x if w not in self.final_stop_words])

        self.dictionary = corpora.Dictionary(self.tokenize_df['tokenized_body'])
        self.dictionary.filter_extremes(no_below=10, no_above=0.4)
        self.BoW_corpus = [self.dictionary.doc2bow(doc) for doc in self.tokenize_df['tokenized_body']]

    def train_model(self, num_topics, num_iters):
        self.ldamallet = gensim.models.wrappers.LdaMallet(self.mallet_path, corpus=self.BoW_corpus, num_topics=num_topics, id2word=self.dictionary, iterations=num_iters)
    
    def print_topics(self, num_words):
        topics = [[(term, round(wt, 3)) for term, wt in self.ldamallet.show_topic(n, topn=20)] for n in range(0, self.ldamallet.num_topics)]
        topics_df = pd.DataFrame([[term for term, wt in topic] for topic in topics], columns = ['Term' + str(i) for i in range(1, 21)],
                                index=['Topic '+ str(t) for t in range(1, self.ldamallet.num_topics + 1)]).T
        return topics_df.head(num_words)
    
    def get_coherence_score(self):
        return gensim.models.CoherenceModel(model=self.ldamallet, corpus=self.BoW_corpus, dictionary=self.dictionary, coherence='u_mass').get_coherence()
    
    def save_all(self, path_model, path_BoW, path_dictionary):
        # Save lda model, BoW and dictionary
        self.ldamallet.save('./Data for CB/ldamallet.gensim')
        pickle.dump(self.BoW_corpus, open("./Data for CB/BoW_corpus.pkl", "wb"))
        self.dictionary.save('./Data for CB/dictionary.gensim')

    def load_all(self, path_model, path_BoW, path_dictionary):
        # Load lda model, BoW and dictionary
        self.ldamallet = gensim.models.wrappers.LdaMallet.load(path_model)
        self.BoW_corpus = pickle.load(open(path_BoW, 'rb'))
        self.dictionary = gensim.corpora.Dictionary.load(path_dictionary)

    def word_cloud(self):
        wc = WordCloud(background_color="white", colormap="Dark2", max_font_size=150, random_state=42)
        plt.rcParams['figure.figsize'] = [20, 15]

        num_topics = len(self.ldamallet.get_topics())
        for i in range(num_topics):
            wc.generate_from_frequencies(dict(self.ldamallet.show_topic(i, topn=20)))
            plt.subplot(5, 4, i+1)
            plt.imshow(wc, interpolation="bilinear")
            plt.axis("off")
            plt.title('Topic ' + str(i))
        plt.show()
    
    def stat_topics(self):
        topic_corpus = [sorted(topics, key=lambda x: x[1], reverse=True)[0][0] for topics in self.ldamallet[self.BoW_corpus]]
        #Build a DataFrame
        dominant_topic_col = sorted(list(set(topic_corpus)))
        doc_count_col = [topic_corpus.count(n) for n in dominant_topic_col]
        total_docs_perc_col = [round(num/sum(doc_count_col), 2) * 100 for num in doc_count_col]
        d = {'Dominant Topic': dominant_topic_col, 'Doc Count': doc_count_col, 'Total Docs Perc': total_docs_perc_col}
        
        return pd.DataFrame(data=d)
    
    def create_topics_weight(self):
        return pd.DataFrame.from_records([{v: round(k * 100, 2) for v, k in row} for row in self.ldamallet[self.BoW_corpus]])


    


In [ ]:
a = LDAmodel(tokenize_df, stop_words_add, stop_words_remove)

In [ ]:
a.load_all(path_model='./Data for CB/ldamallet.gensim', 
           path_BoW='./Data for CB/BoW_corpus.pkl', path_dictionary='./Data for CB/dictionary.gensim')

/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:479: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:479: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


In [ ]:
a.train_model(num_topics=13, num_iters=1)

/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:479: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:479: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


In [ ]:
a.remove_stop_words()